In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.5 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np



In [4]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [6]:
tokens = tokenizer.encode("i hate this, absolutely the worst", return_tensors = "pt")

In [7]:
tokens[0]

tensor([  101,   151, 39487, 10372,   117, 35925, 10563, 10103, 43060,   102])

In [8]:
#tokenizer.decode(tokens[0])

In [9]:
result = model(tokens)

In [10]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 5.0607,  1.6029, -1.0123, -3.0154, -1.8469]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [12]:
torch.argmax(result.logits)
#return the position of the highest value

tensor(0)

In [13]:
int(torch.argmax(result.logits)) + 1
# when it is negative it will return 1
# when it is positive it will return 5
# when igt is neutral it will return 3

1

collect reviews from yelp website and then apply BERT model for sentiment analysis

In [14]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*') # to extract specific components from the web pages that we parse into soup variable, in our case all the classes that contain comment
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results] # to get just the text of the component

In [15]:
soup

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/no-js/,"js");</script><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="en-US" http-equiv="Content-Language"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><link content="#FF1A1A" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" rel="mask-icon" sizes="any"/><link href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico" rel="shortcut icon"/><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>
            window.yelp = window.yelp || {};
      

In [16]:
results

[<p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.</span></p>,
 <p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">The food and service here was really good.  It was more like tapas food than Mexican food!  The drinks were amazing too!</span></p>,
 <p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">Visiting from Texas and decided to give this restaurant a try.  We were pleasantly surprised.  While the margaritas are more like martinis, the food was excellent.  More like a tapas, Mexican fusion.  Great way to try different plates.<br/>The real treat was Chelsea, our waitress.  Took the time to explain the menu and offer suggestions.  Always smiling and very pleasant.  Best service we ha

In [17]:
results[0].text

'Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.'

In [18]:
reviews

['Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.',
 'The food and service here was really good. \xa0It was more like tapas food than Mexican food! \xa0The drinks were amazing too!',
 'Visiting from Texas and decided to give this restaurant a try. \xa0We were pleasantly surprised. \xa0While the margaritas are more like martinis, the food was excellent. \xa0More like a tapas, Mexican fusion. \xa0Great way to try different plates.The real treat was Chelsea, our waitress. \xa0Took the time to explain the menu and offer suggestions. \xa0Always smiling and very pleasant. \xa0Best service we have had in Sydney!',
 'Great atmosphere, attentive service, solid margs, and a Tasty menu. The Brisket Tacos were substantial and delicious. The corn ribs??? \xa0Fawgetaboutit! \xa0Unreal. \xa0Wanted to order another plate.',
 "Don't come here expecting 

In [19]:
reviews[0]

'Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.'

In [20]:
df = pd.DataFrame(np.array(reviews), columns=['review']) # create a datFrame to store the reviews

In [21]:
df

,review
0,Hotter than I imagine myself to be and firing ...
1,The food and service here was really good. It...
2,Visiting from Texas and decided to give this r...
3,"Great atmosphere, attentive service, solid mar..."
4,Don't come here expecting legit Mexican food b...
5,Out of all the restaurants that I tried in Syd...
6,We came here on a Thursday night @ 5pm and by ...
7,The food is fresh and tasty. The scallop cevi...
8,Have been here twice and have absolutely loved...
9,I was pleasantly surprised at what a great job...


In [22]:
df.head() # returns the first 5

,review
0,Hotter than I imagine myself to be and firing ...
1,The food and service here was really good. It...
2,Visiting from Texas and decided to give this r...
3,"Great atmosphere, attentive service, solid mar..."
4,Don't come here expecting legit Mexican food b...


In [23]:
df.tail() # return the last 5

,review
5,Out of all the restaurants that I tried in Syd...
6,We came here on a Thursday night @ 5pm and by ...
7,The food is fresh and tasty. The scallop cevi...
8,Have been here twice and have absolutely loved...
9,I was pleasantly surprised at what a great job...


In [24]:
df.iloc[0]

review    Hotter than I imagine myself to be and firing ...
Name: 0, dtype: object

In [25]:
df['review'].iloc[0]

'Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.'

In [26]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors = "pt")
  result = model(tokens)
  return int(torch.argmax(result.logits)) + 1




In [27]:
sentiment_score(df['review'].iloc[0])

5

In [28]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x))

In [29]:
df

,review,sentiment
0,Hotter than I imagine myself to be and firing ...,5
1,The food and service here was really good. It...,5
2,Visiting from Texas and decided to give this r...,5
3,"Great atmosphere, attentive service, solid mar...",3
4,Don't come here expecting legit Mexican food b...,3
5,Out of all the restaurants that I tried in Syd...,5
6,We came here on a Thursday night @ 5pm and by ...,4
7,The food is fresh and tasty. The scallop cevi...,4
8,Have been here twice and have absolutely loved...,5
9,I was pleasantly surprised at what a great job...,5
